In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from sklearn.preprocessing import StandardScaler
import hdbscan
from scipy import stats
from tqdm import tqdm
from sklearn.cluster import DBSCAN
from trackml.dataset import load_event, load_dataset
from trackml.score import score_event
from IPython.display import clear_output

In [2]:
path_to_train = "../train_sample/train_100_events"
event_prefix = "event000001000"
hits, cells, particles, truth = load_event(os.path.join(path_to_train, event_prefix))

In [3]:
def merge(cl1, cl2): # merge cluster 2 to cluster 1
    d = pd.DataFrame(data={'s1':cl1,'s2':cl2})
    d['N1'] = d.groupby('s1')['s1'].transform('count')
    d['N2'] = d.groupby('s2')['s2'].transform('count')
    maxs1 = d['s1'].max()
    cond = np.where((d['N2'].values>d['N1'].values) & (d['N2'].values<25)) # tìm vị trí hit với nhit của cluster mới > nhits cluster cũ
    s1 = d['s1'].values 
    s1[cond] = d['s2'].values[cond]+maxs1 # gán tất cả các hit đó thuộc về track mới (+maxs1 để tăng label cho track để nó khác ban đầu)
    return s1

def extract_good_hits(truth, submission):
    tru = truth[['hit_id', 'particle_id', 'weight']].merge(submission, how='left', on='hit_id')
    tru['count_both'] = tru.groupby(['track_id', 'particle_id']).hit_id.transform('count')    
    tru['count_particle'] = tru.groupby(['particle_id']).hit_id.transform('count')
    tru['count_track'] = tru.groupby(['track_id']).hit_id.transform('count')
    return tru[(tru.count_both > 0.5*tru.count_particle) & (tru.count_both > 0.5*tru.count_track)]

def fast_score(good_hits_df):
    return good_hits_df.weight.sum()


def analyze_truth_perspective(truth, submission):
    tru = truth[['hit_id', 'particle_id', 'weight']].merge(submission, how='left', on='hit_id')
    tru['count_both'] = tru.groupby(['track_id', 'particle_id']).hit_id.transform('count')    
    tru['count_particle'] = tru.groupby(['particle_id']).hit_id.transform('count')
    tru['count_track'] = tru.groupby(['track_id']).hit_id.transform('count')
    good_hits = tru[(tru.count_both > 0.5*tru.count_particle) & (tru.count_both > 0.5*tru.count_track)]
    score = good_hits.weight.sum()
    
    anatru = tru.particle_id.value_counts().value_counts().sort_index().to_frame().rename({'particle_id':'true_particle_counts'}, axis=1)
    #anatru['true_particle_ratio'] = anatru['true_particle_counts'].values*100/np.sum(anatru['true_particle_counts'])

    anatru['good_tracks_counts'] = np.zeros(len(anatru)).astype(int)
    anatru['good_tracks_intersect_nhits_avg'] = np.zeros(len(anatru))
    anatru['best_detect_intersect_nhits_avg'] = np.zeros(len(anatru))
    for nhit in tqdm(range(4,20)):
        particle_list  = tru[(tru.count_particle==nhit)].particle_id.unique()
        intersect_count = 0
        good_tracks_count = 0
        good_tracks_intersect = 0
        for p in particle_list:
            nhit_intersect = tru[tru.particle_id==p].count_both.max()
            intersect_count += nhit_intersect
            corresponding_track = tru.loc[tru[tru.particle_id==p].count_both.idxmax()].track_id
            leng_corresponding_track = len(tru[tru.track_id == corresponding_track])
            
            if (nhit_intersect >= nhit/2) and (nhit_intersect >= leng_corresponding_track/2):
                good_tracks_count += 1
                good_tracks_intersect += nhit_intersect
        intersect_count = intersect_count/len(particle_list)
        anatru.at[nhit,'best_detect_intersect_nhits_avg'] = intersect_count
        anatru.at[nhit,'good_tracks_counts'] = good_tracks_count
        if good_tracks_count > 0:
            anatru.at[nhit,'good_tracks_intersect_nhits_avg'] = good_tracks_intersect/good_tracks_count
    
    return score, anatru, good_hits

def precision(truth, submission,min_hits):
    tru = truth[['hit_id', 'particle_id', 'weight']].merge(submission, how='left', on='hit_id')
    tru['count_both'] = tru.groupby(['track_id', 'particle_id']).hit_id.transform('count')    
    tru['count_particle'] = tru.groupby(['particle_id']).hit_id.transform('count')
    tru['count_track'] = tru.groupby(['track_id']).hit_id.transform('count')
    #print('Analyzing predictions...')
    predicted_list  = tru[(tru.count_track>=min_hits)].track_id.unique()
    good_tracks_count = 0
    ghost_tracks_count = 0
    fp_weights = 0
    tp_weights = 0
    for t in predicted_list:
        nhit_track = tru[tru.track_id==t].count_track.iloc[0]
        nhit_intersect = tru[tru.track_id==t].count_both.max()
        corresponding_particle = tru.loc[tru[tru.track_id==t].count_both.idxmax()].particle_id
        leng_corresponding_particle = len(tru[tru.particle_id == corresponding_particle])
        if (nhit_intersect >= nhit_track/2) and (nhit_intersect >= leng_corresponding_particle/2): #if the predicted track is good
            good_tracks_count += 1
            tp_weights += tru[(tru.track_id==t)&(tru.particle_id==corresponding_particle)].weight.sum()
            fp_weights += tru[(tru.track_id==t)&(tru.particle_id!=corresponding_particle)].weight.sum()
        else: # if the predicted track is bad
                ghost_tracks_count += 1
                fp_weights += tru[(tru.track_id==t)].weight.sum()
    all_weights = tru[(tru.count_track>=min_hits)].weight.sum()
    precision = tp_weights/all_weights*100
    print('Precision: ',precision,', good tracks:', good_tracks_count,', total tracks:',len(predicted_list),
           ', loss:', fp_weights, ', reco:', tp_weights, 'reco/loss', tp_weights/fp_weights)
    return precision


class Clusterer(object):
    def __init__(self):                        
        self.abc = []
          
    def initialize(self,dfhits):
        self.cluster = range(len(dfhits))
        
    def Hough_clustering(self,dfh,coef,epsilon,min_samples=1,n_loop=180,verbose=True): # [phi_coef,phi_coef,zdivrt_coef,zdivr_coef,xdivr_coef,ydivr_coef]
        merged_cluster = self.cluster
        mm = 1
        stepii = 0.000005
        count_ii = 0
        adaptive_eps_coefficient = 1
        for ii in np.arange(0, n_loop*stepii, stepii):
            count_ii += 1
            for jj in range(2):
                mm = mm*(-1)
                eps_new = epsilon + count_ii*adaptive_eps_coefficient*10**(-5)
                dfh['a1'] = dfh['a0'].values - np.nan_to_num(np.arccos(mm*ii*dfh['rt'].values))
                dfh['sina1'] = np.sin(dfh['a1'].values)
                dfh['cosa1'] = np.cos(dfh['a1'].values)
                ss = StandardScaler()
                dfs = ss.fit_transform(dfh[['sina1','cosa1','zdivrt','zdivr','xdivr','ydivr']].values) 
                #dfs = scale_ignore_nan(dfh[['sina1','cosa1','zdivrt','zdivr','xdivr','ydivr']])
                dfs = np.multiply(dfs, coef)
                new_cluster=DBSCAN(eps=eps_new,min_samples=min_samples,metric='euclidean',n_jobs=4).fit(dfs).labels_
                merged_cluster = merge(merged_cluster, new_cluster)
                if verbose == True:
                    sub = create_one_event_submission(0, hits, merged_cluster)
                    good_hits = extract_good_hits(truth, sub)
                    score_1 = fast_score(good_hits)
                    print('2r0_inverse:', ii*mm ,'. Score:', score_1)
                    #clear_output(wait=True)
        self.cluster = merged_cluster

def create_one_event_submission(event_id, hits, labels):
    sub_data = np.column_stack(([event_id]*len(hits), hits.hit_id.values, labels))
    submission = pd.DataFrame(data=sub_data, columns=["event_id", "hit_id", "track_id"]).astype(int)
    return submission  

def preprocess_hits(h,dz):
    h['z'] =  h['z'].values + dz
    h['r'] = np.sqrt(h['x'].values**2+h['y'].values**2+h['z'].values**2)
    h['rt'] = np.sqrt(h['x'].values**2+h['y'].values**2)
    h['a0'] = np.arctan2(h['y'].values,h['x'].values)
    h['zdivrt'] = h['z'].values/h['rt'].values
    h['zdivr'] = h['z'].values/h['r'].values
    h['xdivr'] = h['x'].values / h['r'].values
    h['ydivr'] = h['y'].values / h['r'].values
    return h

In [4]:
# Clustering by varying 
#model = Clusterer()
#model.initialize(hits) 
c = [1.5,1.5,0.73,0.17,0.027,0.027] #[phi_coef,phi_coef,zdivrt_coef,zdivr_coef,xdivr_coef,ydivr_coef]
min_samples_in_cluster = 1

model = Clusterer()
model.initialize(hits) 
hits_with_dz = preprocess_hits(hits, 0)
model.Hough_clustering(hits_with_dz,coef=c,epsilon=0.0048,min_samples=min_samples_in_cluster,
                       n_loop=300,verbose=True)

submission = create_one_event_submission(0, hits, model.cluster)
print('\n') 

2r0_inverse: -0.0 . Score: 0.054025613
2r0_inverse: 0.0 . Score: 0.054025613
2r0_inverse: -5e-06 . Score: 0.05483661
2r0_inverse: 5e-06 . Score: 0.05708793
2r0_inverse: -1e-05 . Score: 0.058155157
2r0_inverse: 1e-05 . Score: 0.059309196
2r0_inverse: -1.5000000000000002e-05 . Score: 0.060501873
2r0_inverse: 1.5000000000000002e-05 . Score: 0.06297586
2r0_inverse: -2e-05 . Score: 0.064156964
2r0_inverse: 2e-05 . Score: 0.0651218
2r0_inverse: -2.5e-05 . Score: 0.06608248
2r0_inverse: 2.5e-05 . Score: 0.06720762
2r0_inverse: -3.0000000000000004e-05 . Score: 0.06812451
2r0_inverse: 3.0000000000000004e-05 . Score: 0.06914617
2r0_inverse: -3.5000000000000004e-05 . Score: 0.06936906
2r0_inverse: 3.5000000000000004e-05 . Score: 0.07171189
2r0_inverse: -4e-05 . Score: 0.07299629
2r0_inverse: 4e-05 . Score: 0.07498627
2r0_inverse: -4.5e-05 . Score: 0.07635245
2r0_inverse: 4.5e-05 . Score: 0.078414194
2r0_inverse: -5e-05 . Score: 0.08057461
2r0_inverse: 5e-05 . Score: 0.08248128
2r0_inverse: -5.5e-

2r0_inverse: 0.00042500000000000003 . Score: 0.3814863
2r0_inverse: -0.00043000000000000004 . Score: 0.3827098
2r0_inverse: 0.00043000000000000004 . Score: 0.38411516
2r0_inverse: -0.00043500000000000006 . Score: 0.38462043
2r0_inverse: 0.00043500000000000006 . Score: 0.38590568
2r0_inverse: -0.00044 . Score: 0.38691133
2r0_inverse: 0.00044 . Score: 0.38844967
2r0_inverse: -0.00044500000000000003 . Score: 0.3897668
2r0_inverse: 0.00044500000000000003 . Score: 0.39141107
2r0_inverse: -0.00045000000000000004 . Score: 0.39271182
2r0_inverse: 0.00045000000000000004 . Score: 0.39382613
2r0_inverse: -0.00045500000000000006 . Score: 0.39475155
2r0_inverse: 0.00045500000000000006 . Score: 0.3956153
2r0_inverse: -0.00046 . Score: 0.39703155
2r0_inverse: 0.00046 . Score: 0.39831638
2r0_inverse: -0.000465 . Score: 0.39929843
2r0_inverse: 0.000465 . Score: 0.40109333
2r0_inverse: -0.00047000000000000004 . Score: 0.4018165
2r0_inverse: 0.00047000000000000004 . Score: 0.40295652
2r0_inverse: -0.0004

2r0_inverse: -0.0008500000000000001 . Score: 0.49293104
2r0_inverse: 0.0008500000000000001 . Score: 0.49328095
2r0_inverse: -0.0008550000000000001 . Score: 0.49360955
2r0_inverse: 0.0008550000000000001 . Score: 0.4940549
2r0_inverse: -0.0008600000000000001 . Score: 0.4946259
2r0_inverse: 0.0008600000000000001 . Score: 0.49487472
2r0_inverse: -0.0008650000000000001 . Score: 0.4951072
2r0_inverse: 0.0008650000000000001 . Score: 0.4952157
2r0_inverse: -0.0008700000000000001 . Score: 0.49539495
2r0_inverse: 0.0008700000000000001 . Score: 0.4955734
2r0_inverse: -0.000875 . Score: 0.49579412
2r0_inverse: 0.000875 . Score: 0.49615234
2r0_inverse: -0.00088 . Score: 0.49658534
2r0_inverse: 0.00088 . Score: 0.49665725
2r0_inverse: -0.000885 . Score: 0.4969219
2r0_inverse: 0.000885 . Score: 0.49719942
2r0_inverse: -0.0008900000000000001 . Score: 0.49753463
2r0_inverse: 0.0008900000000000001 . Score: 0.49776846
2r0_inverse: -0.0008950000000000001 . Score: 0.49794945
2r0_inverse: 0.0008950000000000

<ipython-input-3-62b85426b902>:106: RuntimeWarning: invalid value encountered in arccos
  dfh['a1'] = dfh['a0'].values - np.nan_to_num(np.arccos(mm*ii*dfh['rt'].values))


2r0_inverse: -0.0009800000000000002 . Score: 0.5051782


...

2r0_inverse: 0.0014900000000000002 . Score: 0.5252999


<ipython-input-3-62b85426b902>:106: RuntimeWarning: invalid value encountered in arccos
  dfh['a1'] = dfh['a0'].values - np.nan_to_num(np.arccos(mm*ii*dfh['rt'].values))


2r0_inverse: -0.001495 . Score: 0.5254206


<ipython-input-3-62b85426b902>:106: RuntimeWarning: invalid value encountered in arccos
  dfh['a1'] = dfh['a0'].values - np.nan_to_num(np.arccos(mm*ii*dfh['rt'].values))


2r0_inverse: 0.001495 . Score: 0.52551883




In [5]:
pr = precision(truth,submission,min_hits=10)

Precision:  88.62100591667675 , good tracks: 3554 , total tracks: 3669 , loss: 0.05139141237123113 , reco: 0.400242505598726 reco/loss 7.78812037130899


In [6]:
pr = precision(truth,submission,min_hits=7)

Precision:  86.6966189831456 , good tracks: 5096 , total tracks: 5563 , loss: 0.0781183681440325 , reco: 0.5090885805147991 reco/loss 6.516887034508396


In [7]:
pr = precision(truth,submission,min_hits=4)

Precision:  76.65909262518073 , good tracks: 5603 , total tracks: 8669 , loss: 0.1638479780478974 , reco: 0.5381298951706412 reco/loss 3.2843242961065444


In [8]:
pr = precision(truth,submission,min_hits=1)

Precision:  54.15145644480604 , good tracks: 6494 , total tracks: 39908 , loss: 0.45848541640179974 , reco: 0.5415145321712771 reco/loss 1.1810943441147836
